### 最终采用

##### 单独算一个语言，避免共线性（最终采用）

In [1]:
import pandas as pd
import numpy as np
import json
import os
from typing import List

# ==========================================
# 配置与路径
# ==========================================
GRAVITY_CSV = "../../data_origin/Gravity/Gravity_csv_V202211/Gravity_V202211.csv"
JSON_PATH = "../country_list.json"

# 修改输出路径：合并为一个文件
OUT_PATH = "../../data/1-4-Language_Index.csv"

def get_target_isos(meta_path: str) -> List[str]:
    """从JSON配置文件加载目标国家ISO代码列表"""
    with open(meta_path, 'r', encoding='utf-8') as f:
        return [c['iso'] for c in json.load(f)['countries']]

def build_merged_language_matrix(csv_path: str, target_isos: List[str]) -> pd.DataFrame:
    """
    处理Gravity数据集，生成合并后的语言指数矩阵
    逻辑：Language_Index = (comlang_off + comlang_ethno) / 2
    """
    # 1. 读取数据
    df = pd.read_csv(csv_path, usecols=['year', 'iso3_o', 'iso3_d', 'comlang_off', 'comlang_ethno'])
    
    # 2. 筛选：特定年份 + 目标国家
    mask = (df['year'] == 2019) & (df['iso3_o'].isin(target_isos)) & (df['iso3_d'].isin(target_isos))
    df_filtered = df[mask].copy() # 使用 copy 避免 SettingWithCopyWarning

    # 3. 【核心修改】计算合并指数 (取平均值)
    # comlang_off 和 comlang_ethno 通常是 0 或 1
    # 结果可能是 0, 0.5, 或 1
    df_filtered['lang_index'] = (df_filtered['comlang_off'] + df_filtered['comlang_ethno']) / 2

    # 4. 矩阵化
    matrix = df_filtered.pivot_table(index='iso3_o', columns='iso3_d', values='lang_index', aggfunc='max')
    
    # 5. 对齐国家列表并填充缺失值
    matrix = matrix.reindex(index=target_isos, columns=target_isos).fillna(0)
    
    # 6. 【对角线处理】
    # 逻辑：自己和自己的语言相似度应该是 0 (最高)
    # 如果你之前的 GDP_Sim 对角线是 0，这里也建议设为 0，保持逻辑统一
    np.fill_diagonal(matrix.values, 0)

    # 排序
    return matrix.sort_index(axis=0).sort_index(axis=1)

if __name__ == "__main__":
    # 确保输出目录存在
    os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
    
    # 执行处理逻辑
    target_countries = get_target_isos(JSON_PATH)
    
    print("正在生成合并后的语言指数矩阵...")
    m_lang_index = build_merged_language_matrix(GRAVITY_CSV, target_countries)
    
    # 保存结果
    m_lang_index.to_csv(OUT_PATH)
    
    print(f"文件已保存至: {OUT_PATH}")
    print(f"矩阵维度: {m_lang_index.shape}")

正在生成合并后的语言指数矩阵...
文件已保存至: ../../data/1-4-Language_Index.csv
矩阵维度: (49, 49)


#### 分开算官方语言和民族语言

In [7]:
import pandas as pd
import numpy as np
import json
import os
from typing import Tuple, List

# ==========================================
# 配置与路径
# ==========================================
GRAVITY_CSV = "../../data_origin/Gravity/Gravity_csv_V202211/Gravity_V202211.csv"
JSON_PATH = "../country_list.json"
OUT_OFF = "../../data/1-4a-Common_Official_Language.csv"
OUT_ETHNO = "../../data/1-4b-Common_Ethno_Language.csv"

def get_target_isos(meta_path: str) -> List[str]:
    """从JSON配置文件加载目标国家ISO代码列表"""
    with open(meta_path, 'r', encoding='utf-8') as f:
        return [c['iso'] for c in json.load(f)['countries']]

def build_dual_language_matrices(csv_path: str, target_isos: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    处理Gravity数据集，生成官方语言(comlang_off)和民族语言(comlang_ethno)矩阵
    参数:
        csv_path: CEPII Gravity CSV文件路径
        target_isos: 目标国家代码列表
    返回:
        (官方语言矩阵, 民族语言矩阵)
    """
    # 仅读取必要列并过滤2019年数据
    df = pd.read_csv(csv_path, usecols=['year', 'iso3_o', 'iso3_d', 'comlang_off', 'comlang_ethno'])
    mask = (df['year'] == 2019) & (df['iso3_o'].isin(target_isos)) & (df['iso3_d'].isin(target_isos))
    df_filtered = df[mask]

    results = []
    # 分别处理官方语言 和民族语言(>9%人口)
    for var in ['comlang_off', 'comlang_ethno']:
        # 矩阵化并对齐49国顺序
        matrix = df_filtered.pivot_table(index='iso3_o', columns='iso3_d', values=var, aggfunc='max')
        matrix = matrix.reindex(index=target_isos, columns=target_isos).fillna(0)
        
        # 对角线置零（国家自身联系不计入网络分析）
        np.fill_diagonal(matrix.values, 0)
        results.append(matrix.sort_index(axis=0).sort_index(axis=1))

    return tuple(results)

if __name__ == "__main__":
    # 确保输出目录存在
    os.makedirs(os.path.dirname(OUT_OFF), exist_ok=True)
    
    # 执行处理逻辑
    target_countries = get_target_isos(JSON_PATH)
    m_official, m_ethno = build_dual_language_matrices(GRAVITY_CSV, target_countries)
    
    # 保存结果
    m_official.to_csv(OUT_OFF)
    m_ethno.to_csv(OUT_ETHNO)
    
    print(f"Layer 4a & 4b processing complete. Shapes: {m_official.shape}")

Layer 4a & 4b processing complete. Shapes: (49, 49)


### 共同口语 存在采样不均和阈值过于严格的问题，故最后没有采用

In [2]:
import pandas as pd
import json
import os

# ==========================================
# 配置 (根据你的实际路径调整的相对路径)
# ==========================================
# 从 code/data_prepare/ 出发：
# ../../ 回到项目根目录 CAMPF_Supplementary_V2
LING_DTA = "../../data_origin/Language/ling_web.dta" 
JSON_PATH = "../country_list.json" # 假设在 code/ 目录下
OUT_FILE = "../../data/1-4-Common_Spoken_Language.csv"

def process_language_layer(dta_path, meta_path):
    """
    处理 Melitz-Toubal 语言数据集，提取共同口语强度 (csl)
    """
    # 1. 加载 49 国列表
    with open(meta_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)['countries']
        target_isos = [c['iso'] for c in meta]

    # 2. 读取 STATA 文件
    # pandas 会自动处理 .dta 格式
    print(f"正在读取数据文件: {dta_path} ...")
    df = pd.read_stata(dta_path)
    
    # 3. 筛选 49 国数据
    # 只保留出发国和目的国都在列表中的记录
    df_filtered = df[df['iso_o'].isin(target_isos) & df['iso_d'].isin(target_isos)]
    
    # 4. 矩阵化
    # csl (Common Spoken Language) 是 0-1 的概率值，直接使用
    m = df_filtered.pivot_table(index='iso_o', columns='iso_d', values='csl', aggfunc='mean')
    
    # 5. 补齐国家并对齐顺序 (确保 49x49 且顺序一致)
    m = m.reindex(index=target_isos, columns=target_isos).fillna(0)
    
    # 6. 对角线处理 (国家自身的交流强度在网络分析中通常设为 0)
    for i in range(len(m)):
        m.iloc[i, i] = 0
        
    return m.sort_index(axis=0).sort_index(axis=1)

# ==========================================
# 执行
# ==========================================
if __name__ == "__main__":
    if os.path.exists(LING_DTA):
        # 确保输出目录存在
        os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
        
        # 处理数据
        lang_matrix = process_language_layer(LING_DTA, JSON_PATH)
        
        # 保存结果
        lang_matrix.to_csv(OUT_FILE)
        print(f"--- 任务成功 ---")
        print(f"Layer 4 (高级口语交流网络) 已保存至: {OUT_FILE}")
        print(f"矩阵维度: {lang_matrix.shape}")
    else:
        print(f"错误：在路径 {os.path.abspath(LING_DTA)} 下找不到原始数据文件。")
        print("请检查相对路径层级是否正确。")

正在读取数据文件: ../../data_origin/Language/ling_web.dta ...
--- 任务成功 ---
Layer 4 (高级口语交流网络) 已保存至: ../../data/1-4-Common_Spoken_Language.csv
矩阵维度: (49, 49)
